## Solución 2

Esta es la solución 2 propuesta para la predicción de cáncer de pulmon.

Esta solución se basa en un algoritmo genético para la selección de características y la optimización de pesos.
El algoritmo se utiliza para encontrar un subconjunto óptimo de variables y sus correspondientes pesos, que maximicen la precisión del modelo.

La idea central es representar cada individuo de la población con dos tipos de genes:
- Un cromosoma binario, donde cada bit indica si una característica (columna) del conjunto de datos será incluida o no en el modelo.
- Un cromosoma de pesos, donde cada gen asociado a una característica seleccionada define su peso en la predicción final.

Cada individuo representa una solución, un clasificador, y tratamos de mejorar nuestra población para obtener el mejor individuo posible.

Para ello se desarrolla el siguiente código:

### Leer los datos

Primero leemos el dataset y lo mostramos para verificar que se ha leído correctamente

In [ ]:
import pandas as pd
import random
import numpy as np

data=pd.read_csv('/content/LungCancer.csv')
data

,Sexo,Edad,Fumador,DedosAmarillos,Ansiedad,Hipertension,EnfermedadCronica,Fatiga,Alergia,Silbidos,ConsumidorAlcohol,Tos,DificultadRespirar,DificultadTragar,DolorPecho,CancerPulmon
0,M,69,0,1,1,0,0,1,0,1,1,1,1,1,1,1
1,M,74,1,0,0,0,1,1,1,0,0,0,1,1,1,1
2,F,59,0,0,0,1,0,1,0,1,0,1,1,0,1,0
3,M,63,1,1,1,0,0,0,0,0,1,0,0,1,1,0
4,F,63,0,1,0,0,0,0,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,F,59,0,1,1,1,0,0,1,1,0,1,0,1,0,1
280,F,59,1,0,0,0,1,1,1,0,0,0,1,0,0,0
281,M,55,1,0,0,0,0,1,1,0,0,0,1,0,1,0
282,M,46,0,1,1,0,0,0,0,0,0,0,0,1,1,0


### Preprocesado de datos

Primero transformamos todos los predictores a variables binarias:
- Para el sexo M lo transformamos a 0 y F a 1.
- Para la edad hacemos dos grupos (1 y 0) basados en menor o mayor que la mediana (que es 62)

In [ ]:
data['Sexo'] = data['Sexo'].replace({'M': 0, 'F': 1})
mediana = data['Edad'].median()
print(mediana)
data['Edad'] = (data['Edad'] >= mediana).astype(int)

62.0


<ipython-input-3-2de7e35470cf>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Sexo'] = data['Sexo'].replace({'M': 0, 'F': 1})


### Separar en train y test el dataset

Separamos el 80% de los datos para train y 20% para test, y guardamos en X todas las columnas menos la que tenemos que predecir (y)

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(data.columns[-1], axis=1)
y = data[data.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Balancear los datos de la clasificación creando instancias sintéticas

Hay muchas mas filas con 'no cancer', así que utilizamos SMOTE para balancear los datos a traves de instancias sintéticas

In [ ]:
from imblearn.over_sampling import SMOTE

s = SMOTE()
X_train, y_train = s.fit_resample(X_train, y_train)

y_train.value_counts()

,count
CancerPulmon,
1,197
0,197


### Algoritmo

Lo primero es crear a los inviduos y una población inicial.

Cada individuo es una combinación de un cromosoma binario y un cromosoma de pesos. Cada cromosoma binario tiene el mismo número de genes que las características del dataset y genera 1 o 0 de forma aleatoria, y el cromosoma de pesos contiene valores aleatorios entre 0 y 1.
Una población es una lista de individuos.

También se crea una función para normalizar los pesos más adelante.

In [ ]:
# Función para crear un individuo
def crear_individuo():
    cromosoma_binario = [random.randint(0, 1) for _ in range(len(X.columns))]
    cromosoma_pesos = [random.uniform(0, 1) for _ in range(len(X.columns))]
    individuo = cromosoma_binario + cromosoma_pesos
    return individuo

# Función que crea una población (lista de x individuos)
def crear_poblacion(num_individuos):
    poblacion = [crear_individuo() for _ in range(num_individuos)]
    return poblacion

# Función auxiliar para normalizar pesos
def normalizar_pesos(pesos):
    suma = sum(pesos)
    if suma == 0:  # Para evitar la división por cero
        return [0] * len(pesos)  # O podrías devolver un arreglo de ceros
    return [peso / suma for peso in pesos]

Después definimos una función para asignar un fitness a cada individuo, que en este caso es como una accuracy (como de bien predicen?)

Para calcularlo, se seleccionan las características que tienen un valor "1" en el cromosoma binario, se normalizan los pesos seleccionados, y se realiza una predicción ponderada. La función de fitness compara las predicciones con las etiquetas reales para calcular la precisión del individuo.

La función recorre cada individuo en la población y sigue los siguientes pasos:

- Divide al individuo en las dos partes (Cromosoma Binario y del Cromosoma de Pesos)

- Usando el cromosoma binario, se identifican las columnas a mantener en el conjunto de datos (las características marcadas con 1 en el cromosoma).

- Se filtran los pesos correspondientes a esas columnas, de modo que solo se mantengan los pesos asociados a las características seleccionadas.

- Para que los pesos se utilicen correctamente en la predicción, se normalizan de forma que su suma sea igual a 1.

- Con las columnas seleccionadas y sus pesos normalizados, se crea un nuevo DataFrame que contiene solo estas columnas del conjunto de datos original y cada columna se multiplica por su peso correspondiente.

- Se obtiene una nueva columna llamada 'suma' al DataFrame. Esta columna representa la suma ponderada de las características seleccionadas, obtenida sumando los valores ponderados de cada fila.

- Para hacer la predicción de cada fila, se define un threshold de 0.5: si el valor de la columna 'suma' es mayor que 0.5, se asigna una predicción de 1 (positivo para cáncer); si es menor o igual, la predicción es 0 (negativo para cáncer).

- Finalmente, la precisión del individuo se calcula comparando sus predicciones con la columna objetivo.
La precisión se calcula como la proporción de predicciones correctas sobre el total de predicciones realizadas y se almacena en fitness_values.

Esta función devuelve un array con el fitness de cada individuo, permitiendo evaluar y seleccionar aquellos individuos con mejor precisión en sus predicciones.

In [ ]:
# Función para calcular el fitness (accuracy) de cada individuo en la población
def calcular_fitness_poblacion(poblacion, data, y):
    fitness_values = []
    for i in range(len(poblacion)):
        # Obtener el individuo y dividir en cromosoma binario y cromosoma de pesos
        test = poblacion[i]
        size = int(len(test) / 2)
        cromosoma_binario = test[:size]
        cromosoma_pesos = test[size:]

        # Filtrar columnas y pesos a mantener según cromosoma binario
        columnas_a_mantener = [col for col, keep in zip(data.columns, cromosoma_binario) if keep == 1]
        valores_mantener = [peso for peso, keep in zip(cromosoma_pesos, cromosoma_binario) if keep == 1]

        if len(columnas_a_mantener) == 0 or len(valores_mantener) == 0:
            print(f"Individuo {i+1}: No se seleccionaron columnas. Fitness no calculado.")
            continue

        # Normalizar los pesos para que sumen 1
        suma_total = sum(valores_mantener)
        normalizada = [x / suma_total for x in valores_mantener]

        # Crear un dataframe con solo las columnas necesarias
        data_crom = data[columnas_a_mantener].copy()

        # Multiplicar el dataframe por el vector de pesos normalizado
        df_multiplicado = data_crom.mul(normalizada, axis=1)

        # Añadir una columna 'suma' que contiene la suma ponderada de las filas
        data_crom['suma'] = df_multiplicado.sum(axis=1)

        # Hacer la predicción usando un threshold de 0.5
        data_crom['pred'] = data_crom['suma'].apply(lambda x: 1 if x > 0.5 else 0)

        # Calcular el accuracy comparando con la columna objetivo
        accuracy = sum(data_crom['pred'] == y) / len(data_crom)

        fitness_values.append(accuracy)

    return fitness_values

Lo siguiente es una función que nos permite seleccionar a los mejores individuos, probamos diferentes maneras pero al final nos quedamos con un torneo simple de tamaño a definir al llamar a la función, de manera que se asegura que los individuos con mejores valores de fitness tengan más probabilidad de ser seleccionados. La función sigue estos pasos:

- El número de ganadores se define igual al tamaño de la población.

- Mientras no se haya alcanzado el número necesario de ganadores, se repite el siguiente proceso:
  - Se selecciona aleatoriamente un grupo de individuos de la población (según el tamaño tamaño_torneo).
  - Dentro del grupo seleccionado, se elige al individuo con el valor de fitness más alto.
  - El índice del ganador se añade a la lista de ganadores.

- Una vez completado el proceso, la función devuelve una lista de individuos ganadores basada en sus índices en la población original.

In [ ]:
# Función para seleccionar a los mejores individuos
def seleccion_torneo(poblacion, fitness_values, tamaño_torneo):
    num_ganadores = len(poblacion)
    ganadores = []

    while len(ganadores) < num_ganadores:
        # Seleccionamos al azar 'tamaño_torneo' individuos
        seleccionados = random.sample(range(len(poblacion)), tamaño_torneo)
        # Obtenemos el índice del mejor individuo de los seleccionados
        ganador = max(seleccionados, key=lambda idx: fitness_values[idx])
        ganadores.append(ganador)  # Añadir el índice del ganador al conjunto

    # Convertimos los índices de ganadores a individuos
    return [poblacion[i] for i in ganadores]

Definimos una función para cruzar a los individuos de nuestra población e introducir variabilidad. Para esto también probamos otros tipos de cruces, pero el que decidimos usar hace:

- Divide los Cromosomas: Los individuos individuo1 e individuo2 (padres) se dividen en dos partes, la binaria y la de los pesos.

- Se eligen dos puntos de cruce aleatorios, uno para cada mitad (binaria y pesos).

- Se combinan las partes de ambos padres usando los puntos de cruce:
    - Hijo 1: toma los elementos de individuo1 hasta el punto de cruce en el cromosoma binario, luego se complementa con individuo2. La segunda mitad sigue el mismo esquema.
    - Hijo 2: toma los elementos de individuo2 hasta el punto_cruce y se complementa con individuo1 después del cruce, con la segunda mitad construida de forma similar.

- La función devuelve dos nuevos individuos, hijo1 y hijo2, que contienen una combinación de los genes de ambos padres.

In [ ]:
# Función que cruza a dos individuos (padres), para obtener otros dos (hijos)
def cruce(individuo1, individuo2):
    # Encuentra el punto de cruce
    lista1_1=individuo1[:len(individuo1) // 2]
    lista1_2=individuo1[len(individuo1) // 2:]
    lista2_1=individuo2[:len(individuo2) // 2]
    lista2_2=individuo2[len(individuo2) // 2:]

    punto_cruce = random.randint(0, len(lista1_1) -1)  # Desde la mitad hasta el final
    punto_cruce2 = random.randint(0, len(lista2_2) -1 )

    # Crear hijos
    hijo1 = lista1_1[:punto_cruce] + lista2_1[punto_cruce:] + lista1_2[:punto_cruce2] + lista2_2[punto_cruce2:]
    hijo2 = lista2_1[:punto_cruce] + lista1_1[punto_cruce:] + lista2_2[:punto_cruce2] + lista1_2[punto_cruce2:]  # Parte de individuo1 y parte de individuo2
    return hijo1, hijo2

Para la mutación, probamos con diferentes parametros que se fueran reduciendo conforme pasaban las generaciones para empezar con más exploración y luego hacer más explotación, pero como la solución mejor y se estanca muy rápido, tampoco hacía falta disminuir esa mutación.

Así que nuestra función mutar aplica cambios en los cromosomas con una probabilidad dada (tasa_mutacion). Este es el proceso que sigue:

- El individuo se divide en dos partes: el cromosoma binario y el cromosoma de pesos.

- Para cada gen binario (0 o 1) en la primera mitad, si se cumple la probabilidad de mutación (random.random() < tasa_mutacion), si el valor es 1 se convierte en 0 y viceversa.

- Para los genes en la segunda mitad (cromosoma de pesos), se aplica una modificación aleatoria en un rango proporcional a su valor actual. Esto se hace sumando o restando un pequeño valor de hasta un 10% de su valor original, creando una leve variación en los pesos. La mutación se asegura de que los valores no sean negativos.

- Tras la mutación, los pesos se normalizan.

La función devuelve el individuo mutado, listo para la próxima generación con leves ajustes genéticos, para aportar variación e intentar evitar que la población se estanque en soluciones locales.

In [ ]:
# Función para mutar a un individuo
def mutar(individuo, tasa_mutacion):
    size = len(individuo) // 2

    # Mutamos el cromosoma binario
    for i in range(size):
        if random.random() < tasa_mutacion:
            individuo[i] = 1 - individuo[i]  # Invertir binario

    # Mutamos el cromosoma de pesos
    for i in range(size, len(individuo)):
        if random.random() < tasa_mutacion:
            # Mutación de pesos
            nuevo_peso = individuo[i] + random.uniform(-0.1 * individuo[i], 0.1 * individuo[i]) # Añadir ruido
            individuo[i] = max(nuevo_peso, 0)  # Asegurar que no sea negativo

    # Normalizar pesos después de la mutación
    individuo[size:] = normalizar_pesos(individuo[size:])  # Normalizar solo los pesos

    return individuo

Por último definimos una función para ponerlo todo junto en orden, y, a partir de una generació, obtener la siguiente. Y otra funcioncita para monitorizar, que imprime una población.

Los pasos para crear la nueva generación son:

- Selección de Padres: Selecciona pares de padres de la población mediante torneo, basándose en los valores de fitness.
- Cruce: Para cada par de padres seleccionados, se realiza un cruce para generar dos hijos, combinando características de ambos.
- Mutación: Los hijos generados se someten a mutación, introduciendo pequeñas variaciones según la tasa de mutación especificada.
- Nueva Generación: Los hijos mutados se añaden a la nueva generación hasta alcanzar el tamaño de la población original.

Finalmente, se devuelve esta nueva generación, lista para la próxima iteración.

In [ ]:
# Poner los pasos juntos para crear una nueva generación
def crear_nueva_generacion(poblacion, fitness_values, tamaño_torneo, tasa_mutacion):
    nueva_generacion = []
    # Generar nuevos individuos hasta llenar la nueva generación
    while len(nueva_generacion) < len(poblacion):
        # Selección de dos padres
        padres = seleccion_torneo(poblacion, fitness_values, tamaño_torneo)

        # Cruce para crear hijos
        hijo1, hijo2 = cruce(padres[0], padres[1])

        # Mutación de los hijos
        hijo1_mutado = mutar(hijo1, tasa_mutacion)
        hijo2_mutado = mutar(hijo2, tasa_mutacion)

        # Añadir los hijos a la nueva generación
        nueva_generacion.append(hijo1_mutado)
        nueva_generacion.append(hijo2_mutado)

    # Nueva generación tenga el mismo tamaño que la original
    return nueva_generacion[:len(poblacion)]

def print_poblacion(poblacion):
    for i, individuo in enumerate(poblacion):
        print(f"Individuo {i + 1}: {individuo}")

### Pruebas

Hemos hecho muchas pruebas para ver que tal lo hacía esta solución, aquí reflejamos algunas, cambiando estos parámetros:

- num_generaciones = numero de generaciones
- tamaño_poblacion = tamaño de la población inicial
- tamaño_torneo = tamaño de los torneos
- tasa_mutacion = tasa de mutación

Realmente lo que nos interesa mirar es la accuracy del mejor individuo de la población final, que es lo que querríamos utilizar como clasificador.

In [ ]:
# Prueba 1
num_generaciones = 200
tamaño_poblacion = 300
tamaño_torneo = 5
tasa_mutacion = 0.1

# Inicializar la población
poblacion = crear_poblacion(tamaño_poblacion)

for generacion in range(num_generaciones):
    # Evaluar fitness
    fitnesses = calcular_fitness_poblacion(poblacion, X_train, y_train)

    # Seleccionar los mejores individuos
    seleccionados = seleccion_torneo(poblacion, fitnesses, tamaño_torneo)

    # Crear nueva generación
    nueva_generacion = []
    while len(nueva_generacion) < tamaño_poblacion:
        padre1, padre2 = random.sample(seleccionados, 2)  # Seleccionar dos padres aleatorios
        hijo1, hijo2 = cruce(padre1, padre2)
        nueva_generacion.append(mutar(hijo1, tasa_mutacion))
        if len(nueva_generacion) < tamaño_poblacion:
            nueva_generacion.append(mutar(hijo2, tasa_mutacion))

    # Reemplazar la población anterior con la nueva generación
    poblacion = nueva_generacion

    # Imprimir resultados de esta generación
    mejor_fitness = max(fitnesses)
    promedio_fitness = sum(fitnesses) / len(fitnesses)
    print(f"Generación {generacion + 1}: Promedio fitness: {promedio_fitness:.4f}")

# Resultados finales
print("Simulación completada. Resultados finales:")
print(f"Población final: ")
print_poblacion(poblacion)
print(f"Accuracy del mejor individuo: ")
print(max(calcular_fitness_poblacion(poblacion, X_test, y_test)))

Generación 1: Promedio fitness: 0.7870
Generación 2: Promedio fitness: 0.8193
Generación 3: Promedio fitness: 0.8334
Generación 4: Promedio fitness: 0.8474
Generación 5: Promedio fitness: 0.8526
Generación 6: Promedio fitness: 0.8588
Generación 7: Promedio fitness: 0.8608
Generación 8: Promedio fitness: 0.8683
Generación 9: Promedio fitness: 0.8709
Generación 10: Promedio fitness: 0.8717
Generación 11: Promedio fitness: 0.8711
Generación 12: Promedio fitness: 0.8743
Generación 13: Promedio fitness: 0.8759
Generación 14: Promedio fitness: 0.8748
Generación 15: Promedio fitness: 0.8732
Generación 16: Promedio fitness: 0.8780
Generación 17: Promedio fitness: 0.8738
Generación 18: Promedio fitness: 0.8784
Generación 19: Promedio fitness: 0.8770
Generación 20: Promedio fitness: 0.8784
Generación 21: Promedio fitness: 0.8795
Generación 22: Promedio fitness: 0.8806
Generación 23: Promedio fitness: 0.8789
Generación 24: Promedio fitness: 0.8809
Generación 25: Promedio fitness: 0.8806
Generació

In [ ]:
# Prueba 2 -> Aumentando el tamaño de la población
num_generaciones = 200
tamaño_poblacion = 500
tamaño_torneo = 5
tasa_mutacion = 0.1

# Inicializar la población
poblacion = crear_poblacion(tamaño_poblacion)

for generacion in range(num_generaciones):
    # Evaluar fitness
    fitnesses = calcular_fitness_poblacion(poblacion, X_train, y_train)

    # Seleccionar los mejores individuos
    seleccionados = seleccion_torneo(poblacion, fitnesses, tamaño_torneo)

    # Crear nueva generación
    nueva_generacion = []
    while len(nueva_generacion) < tamaño_poblacion:
        padre1, padre2 = random.sample(seleccionados, 2)  # Seleccionar dos padres aleatorios
        hijo1, hijo2 = cruce(padre1, padre2)
        nueva_generacion.append(mutar(hijo1, tasa_mutacion))
        if len(nueva_generacion) < tamaño_poblacion:
            nueva_generacion.append(mutar(hijo2, tasa_mutacion))

    # Reemplazar la población anterior con la nueva generación
    poblacion = nueva_generacion

    # Imprimir resultados de esta generación
    mejor_fitness = max(fitnesses)
    promedio_fitness = sum(fitnesses) / len(fitnesses)
    print(f"Generación {generacion + 1}: Promedio fitness: {promedio_fitness:.4f}")

# Resultados finales
print("Simulación completada. Resultados finales:")
print(f"Población final: ")
print_poblacion(poblacion)
print(f"Accuracy del mejor individuo: ")
print(max(calcular_fitness_poblacion(poblacion, X_test, y_test)))

Generación 1: Promedio fitness: 0.7777
Generación 2: Promedio fitness: 0.8149
Generación 3: Promedio fitness: 0.8285
Generación 4: Promedio fitness: 0.8407
Generación 5: Promedio fitness: 0.8505
Generación 6: Promedio fitness: 0.8574
Generación 7: Promedio fitness: 0.8617
Generación 8: Promedio fitness: 0.8638
Generación 9: Promedio fitness: 0.8674
Generación 10: Promedio fitness: 0.8696
Generación 11: Promedio fitness: 0.8741
Generación 12: Promedio fitness: 0.8719
Generación 13: Promedio fitness: 0.8750
Generación 14: Promedio fitness: 0.8780
Generación 15: Promedio fitness: 0.8774
Generación 16: Promedio fitness: 0.8792
Generación 17: Promedio fitness: 0.8790
Generación 18: Promedio fitness: 0.8785
Generación 19: Promedio fitness: 0.8818
Generación 20: Promedio fitness: 0.8820
Generación 21: Promedio fitness: 0.8825
Generación 22: Promedio fitness: 0.8835
Generación 23: Promedio fitness: 0.8833
Generación 24: Promedio fitness: 0.8856
Generación 25: Promedio fitness: 0.8889
Generació

In [ ]:
# Prueba 3 -> Reduciendo el tamaño del torneo
num_generaciones = 200
tamaño_poblacion = 300
tamaño_torneo = 3
tasa_mutacion = 0.1

# Inicializar la población
poblacion = crear_poblacion(tamaño_poblacion)

for generacion in range(num_generaciones):
    # Evaluar fitness
    fitnesses = calcular_fitness_poblacion(poblacion, X_train, y_train)

    # Seleccionar los mejores individuos
    seleccionados = seleccion_torneo(poblacion, fitnesses, tamaño_torneo)

    # Crear nueva generación
    nueva_generacion = []
    while len(nueva_generacion) < tamaño_poblacion:
        padre1, padre2 = random.sample(seleccionados, 2)  # Seleccionar dos padres aleatorios
        hijo1, hijo2 = cruce(padre1, padre2)
        nueva_generacion.append(mutar(hijo1, tasa_mutacion))
        if len(nueva_generacion) < tamaño_poblacion:
            nueva_generacion.append(mutar(hijo2, tasa_mutacion))

    # Reemplazar la población anterior con la nueva generación
    poblacion = nueva_generacion

    # Imprimir resultados de esta generación
    mejor_fitness = max(fitnesses)
    promedio_fitness = sum(fitnesses) / len(fitnesses)
    print(f"Generación {generacion + 1}: Promedio fitness: {promedio_fitness:.4f}")

# Resultados finales
print("Simulación completada. Resultados finales:")
print(f"Población final: ")
print_poblacion(poblacion)
print(f"Accuracy del mejor individuo: ")
print(max(calcular_fitness_poblacion(poblacion, X_test, y_test)))

Generación 1: Promedio fitness: 0.7859
Generación 2: Promedio fitness: 0.8024
Generación 3: Promedio fitness: 0.8119
Generación 4: Promedio fitness: 0.8229
Generación 5: Promedio fitness: 0.8392
Generación 6: Promedio fitness: 0.8419
Generación 7: Promedio fitness: 0.8527
Generación 8: Promedio fitness: 0.8525
Generación 9: Promedio fitness: 0.8590
Generación 10: Promedio fitness: 0.8604
Generación 11: Promedio fitness: 0.8626
Generación 12: Promedio fitness: 0.8665
Generación 13: Promedio fitness: 0.8685
Generación 14: Promedio fitness: 0.8703
Generación 15: Promedio fitness: 0.8683
Generación 16: Promedio fitness: 0.8690
Generación 17: Promedio fitness: 0.8690
Generación 18: Promedio fitness: 0.8702
Generación 19: Promedio fitness: 0.8716
Generación 20: Promedio fitness: 0.8702
Generación 21: Promedio fitness: 0.8712
Generación 22: Promedio fitness: 0.8721
Generación 23: Promedio fitness: 0.8740
Generación 24: Promedio fitness: 0.8702
Generación 25: Promedio fitness: 0.8745
Generació

In [ ]:
# Prueba 4 -> Aumentando la tasa de mutación
num_generaciones = 200
tamaño_poblacion = 300
tamaño_torneo = 5
tasa_mutacion = 0.3

# Inicializar la población
poblacion = crear_poblacion(tamaño_poblacion)

for generacion in range(num_generaciones):
    # Evaluar fitness
    fitnesses = calcular_fitness_poblacion(poblacion, X_train, y_train)

    # Seleccionar los mejores individuos
    seleccionados = seleccion_torneo(poblacion, fitnesses, tamaño_torneo)

    # Crear nueva generación
    nueva_generacion = []
    while len(nueva_generacion) < tamaño_poblacion:
        padre1, padre2 = random.sample(seleccionados, 2)  # Seleccionar dos padres aleatorios
        hijo1, hijo2 = cruce(padre1, padre2)
        nueva_generacion.append(mutar(hijo1, tasa_mutacion))
        if len(nueva_generacion) < tamaño_poblacion:
            nueva_generacion.append(mutar(hijo2, tasa_mutacion))

    # Reemplazar la población anterior con la nueva generación
    poblacion = nueva_generacion

    # Imprimir resultados de esta generación
    mejor_fitness = max(fitnesses)
    promedio_fitness = sum(fitnesses) / len(fitnesses)
    print(f"Generación {generacion + 1}: Promedio fitness: {promedio_fitness:.4f}")

# Resultados finales
print("Simulación completada. Resultados finales:")
print(f"Población final: ")
print_poblacion(poblacion)
print(f"Accuracy del mejor individuo: ")
print(max(calcular_fitness_poblacion(poblacion, X_test, y_test)))

Generación 1: Promedio fitness: 0.7810
Generación 2: Promedio fitness: 0.8047
Generación 3: Promedio fitness: 0.8142
Generación 4: Promedio fitness: 0.8172
Generación 5: Promedio fitness: 0.8256
Generación 6: Promedio fitness: 0.8249
Generación 7: Promedio fitness: 0.8283
Generación 8: Promedio fitness: 0.8282
Generación 9: Promedio fitness: 0.8312
Generación 10: Promedio fitness: 0.8332
Generación 11: Promedio fitness: 0.8370
Generación 12: Promedio fitness: 0.8365
Generación 13: Promedio fitness: 0.8362
Generación 14: Promedio fitness: 0.8388
Generación 15: Promedio fitness: 0.8422
Generación 16: Promedio fitness: 0.8371
Generación 17: Promedio fitness: 0.8348
Generación 18: Promedio fitness: 0.8385
Generación 19: Promedio fitness: 0.8361
Generación 20: Promedio fitness: 0.8369
Generación 21: Promedio fitness: 0.8383
Generación 22: Promedio fitness: 0.8338
Generación 23: Promedio fitness: 0.8429
Generación 24: Promedio fitness: 0.8385
Generación 25: Promedio fitness: 0.8399
Generació

Con esta solución se obtienen resultados de accuracy un poco peores que con las otras dos que proponemos, pero mucho más rápido.

El mejor resultado lo obtenemos con estos parámetros (aumentando la tasa de mutación):

- num_generaciones = 200
- tamaño_poblacion = 300
- tamaño_torneo = 5
- tasa_mutacion = 0.3

Con una accuracy de 91,22%